<font color=gray>ADS Sample Notebook.

Copyright (c) 2020 Oracle, Inc. All rights reserved. Licensed under the Universal Permissive License v 1.0 as shown at https://oss.oracle.com/licenses/upl.
</font>

***
# Getting Started with Oracle Cloud Infrastructure Data Science
<p style="margin-left:10%; margin-right:10%;">by the <font color=teal> Oracle Cloud Infrastructure Data Science Service Team </font></p>

***

## Service Overview

Welcome to Oracle Cloud Infrastructure Data Science Service.

Oracle Cloud Infrastructure Data Science service is a fully managed platform for data science teams to build, train, and manage machine learning models using Oracle Cloud Infrastructure (OCI).

The Data Science service:

* Provides data scientists with a collaborative, project-driven workspace.
* Enables self-service access to infrastructure for data science workloads.
* Includes Python-centric tools, libraries, and packages developed by the open-source community and the [Oracle Accelerated Data Science Library](https://docs.cloud.oracle.com/en-us/iaas/tools/ads-sdk/latest/index.html), which supports the end-to-end lifecycle of predictive models:
    * Data acquisition, profiling, preparation, and visualization.
    * Feature engineering.
    * Model training.
    * Model evaluation, explanation, and interpretation.
    * Model storage through the [Model Catalog](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/manage-models.htm). 
    * Model deployment.
* Integrates with the rest of the OCI services, including [Oracle Functions](https://docs.cloud.oracle.com/en-us/iaas/Content/Functions/Concepts/functionsoverview.htm), [Data Flow](https://docs.cloud.oracle.com/en-us/iaas/data-flow/using/dfs_data_flow.htm), [Autonomous Data Warehouse](https://docs.cloud.oracle.com/en-us/iaas/Content/Database/Concepts/adboverview.htm), [Streaming](https://docs.cloud.oracle.com/en-us/iaas/Content/Streaming/Concepts/streamingoverview.htm), [Vault](https://docs.cloud.oracle.com/en-us/iaas/Content/KeyManagement/Concepts/keyoverview.htm), [Logging](https://docs.cloud.oracle.com/en-us/iaas/Content/Logging/Concepts/loggingoverview.htm#loggingoverview), and [Object Storage](https://docs.cloud.oracle.com/en-us/iaas/Content/Object/Concepts/objectstorageoverview.htm).
* Helps data scientists concentrate on methodology and domain expertise to deliver more models to production.

For more details, see the [Data Science documentation](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm).

---

## Overview

This **[Oracle Database](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/use-notebook-sessions.htm#conda_understand_environments)** conda environment enables you to work seamlessly with Oracle Databases using the `ADS Connector`, `SQLAlchemy` and `ipython-sql`. Use a notebook to create ETL jobs, batch transform data, and perform database queries. The ADS Connector provides a uniform interface to connect to databases. The conda includes support for Oracle Database, MySQL and SQLite. The `ipython-sql` magic extension (`%sql` or `%%sql`) allows you to connect to various databases and issue SQL commands directly in a notebook cell. You can build ETL jobs or integrate the results into your Python code. The examples in this notebook emphasize how to connect to an Oracle Database and access data.

---

**Important:**

Placeholder text for required values are surrounded by angle brackets that must be removed when adding the indicated content. For example, when adding a database name to `database_name = "<database_name>"` would become `database_name = "production"`.

---

## Prerequisites:
- Experience with a specific topic: Novice
- Professional experience: None

---

## Objectives:

- <a href='#authentication'>Understanding Authentication to Oracle Cloud Infrastructure Resources from a Notebook Session</a>
    - <a href='#resource_principals'>Authentication with Resource Principals</a>
        - <a href='#resource_principals_ads'>Resource Principals Authentication using the ADS SDK</a>
        - <a href='#resource_principals_oci'>Resource Principals Authentication using the OCI SDK</a>
        - <a href='#resource_principals_cli'>Resource Principals Authentication using the OCI CLI</a> 
    - <a href='#api_keys'>Authentication with API Keys</a>
- <a href='#conda'>Oracle Database Conda Environment</a>
    - <a href='#conda_overview'>Overview</a>
    - <a href='#conda_libraries'>Principal Conda Libraries</a>
    - <a href='#conda_configuration'>Configuration</a>
- <a href='#sql'>`ipython-sql` Example</a>
    - <a href='#sql_connect'>Connect a Database to the `ipython-sql` Module</a>
    - <a href='#sql_DML'>Data Manipulation Language Commands</a>
    - <a href='#sql_DQL'>Data Query Language Commands</a>
    - <a href='#sql_var'>Variable Bindings</a>
- <a href='#ref'>References</a> 

---

In [ ]:
import logging
import warnings

from ads import set_auth
from ads import set_documentation_mode
from oci.auth.signers import get_resource_principals_signer
from oci.data_science import DataScienceClient
from os import popen

set_documentation_mode(False)
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

<a id='authentication'></a>
# Understanding Authentication to Oracle Cloud Infrastructure Resources from a Notebook Session

When working within a notebook session, the `datascience` user is used. This user does not have an OCI Identity and Access Management (IAM) identity, so it has no access to the OCI API. To access OCI resources, including Data Science projects, models and any other OCI service resources from the notebook environment, you must configure either resource principals or API keys. For most applications, the resource principal is the recommended approach.

<a id='resource_principals'></a>
## Authentication with Resource Principals

Data Science enables easy and secure authentication using the notebook session's resource principal to access other OCI resources, including Data Science projects and models. These steps show you how to use your notebook session's resource principal.

In advance, a tenancy administrator must write policies to grant permissions to the resource principal to access other OCI resources, see [Manually Configuring Your Tenancy for Data Science](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/configure-tenancy.htm).

There are two methods to configure the notebook to use resource principals, and they are the `ads` or `oci` libraries. While both these libraries provide the required authentication, the `ads` library is has been specifically designed for easy operation within a Data Science notebook session.

If you don't want to take on these library dependencies, you can use the `oci` command from the command line.

For more details on using resource principals in the Data Science service, see the [ADS Configuration documentation](https://docs.cloud.oracle.com/en-us/iaas/tools/ads-sdk/latest/user_guide/configuration/configuration.html#) and [Authenticating to the Oracle Cloud Infrastructure APIs from a Notebook Session](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/use-notebook-sessions.htm#topic_kxj_znw_pkb).

<a id='resource_principals_ads'></a>
### Resource Principals Authentication using the ADS SDK

The `set_auth()` method sets the proper authentication mechanism for ADS. ADS uses the `oci` SDK to access resources like the model catalog or Object Storage.

Within a notebook session, configure the use of a resource principal for the ADS SDK by running this in a notebook cell:

In [ ]:
set_auth(auth='resource_principal') 

<a id='resource_principals_oci'></a>
### Resource Principals Authentication using the OCI SDK

Within your notebook session, the `oci` library can use the resource principal.  This cell demonstrates how to make a basic connection using the default settings.

In [ ]:
resource_principal = get_resource_principals_signer() 
dsc = DataScienceClient(config={}, signer=resource_principal)

<a id='resource_principals_cli'></a>
### Resource Principals Authentication using the OCI CLI

Within a notebook session, the OCI CLI can be used to configure the resource principal using the `--auth=resource_principal` option. For example:

In [ ]:
cmd = "oci data-science project get --project-id=$PROJECT_OCID --auth=resource_principal 2>&1"
print(popen(cmd).read())

If the resource principal is correctly configured, a message similar to the following will be printed.

```
{
"data": {
"compartment-id": "ocid1.compartment.oc1..aaaaaaaafl3avkal72rrwuy4m5rumpwh7r4axejjwq5hvwjy4h4uoyi7kzyq",
"created-by": "ocid1.user.oc1..aaaaaaaabfrlcbiyvjmjvgh3ns6trdyoewxytqywwta3yqmy3ah3fa3uw76q",
"defined-tags": {},
"description": "my favorite demo project\n",
"display-name": "jr-demo-project",
"freeform-tags": {},
"id": "ocid1.datascienceproject.oc1.iad.aaaaaaaappvg4tp5kmbkurcyeghxaqmaknw3s5yh2oxcvfrvjeaadinsng6q",
"lifecycle-state": "ACTIVE",
"time-created": "2019-11-14T22:29:06.870000+00:00"
},
"etag": "b4d66fb733748f3454206d5de6b9acb3634edc804b2ad1997bd69dc676035a89"
}
```

<a id='api_keys'></a>
## Authentication with API Keys

If resource principals are not explicitly used, API Keys are used by default. For some use cases, you may want to set up API keys, see the instructions in the api_keys.ipynb example notebook.


<a id='conda'></a>
# Oracle Database Conda Environment

<a id='conda_overview'></a>
## Overview

This **[Oracle Database](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/use-notebook-sessions.htm#conda_understand_environments)** conda environment enables you to work seamlessly with Oracle Databases using the `ADS Connector`, `SQLAlchemy` and `ipython-sql`. Use a notebook to create ETL jobs, batch transform data, and perform database queries. The ADS Connector provides a uniform interface to connect to databases. The conda includes support for Oracle, MySQL, and SQLite. `ipython-sql` adds JupyterLab magic (`%sql` or `%%sql`) to connect to various databases and issue SQL commands directly in a notebook cell. Build ETL jobs or integrate the results into your python code.

You can access notebook examples for this conda environment in JupyterLab from the Launcher tab by clicking **Notebook Examples**. Then you can select one of the notebook examples that are available for all of the conda environments installed in your notebook session. The name of each notebook example contains the path `/notebook/<installed_conda_name>/<notebook_name>.ipynb`. For example, the Data Exploration and Manipulation conda is named `explvX` (`X` is the version of the conda environment).

For a description of each notebook example, see [Overview of the Notebook Examples](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/use-notebook-sessions.htm#overview_of_the_notebook_examples). The notebook examples for the Data Exploration and Manipulation environment emphasize the use of the ADS (https://docs.cloud.oracle.com/en-us/iaas/tools/ads-sdk/latest/index.html) library for a variety of data acccess, transformation, and visualization tasks.

<a id='conda_libraries'></a>
## Principal Conda Libraries

1. `cx-Oracle`: A Python extension module that enables access to Oracle Database.
2. `sqlite3`: Provides an sqlite3 driver.
3. `ipython-sql`: Adds JupyterLab magic (%sql or %%sql) to connect to various databases and issue SQL commands directly in a notebook cell.
4. `mysql-connector-python`: A self-contained Python driver for communicating with MySQL servers.
5. `SQLAlchemy`: The Python SQL toolkit and Object Relational Mapper that gives application developers the full power and flexibility of SQL.

<a id='conda_configuration'></a>
## Configuration

* For detailed instructions to connect to an Oracle Autonomous Database instance, see the `autonomous_database.ipynb` example notebook.
* To understand how to use the OCI Vault service to securely manage your credentials, see the `vault.ipynb` example notebook.

<a id='sql'></a>
# `ipython-sql` Example

`ipython-sql` uses a number of ipython magic commands to interact directly with the database. The following sections will cover the following magic commands
* `load_ext sql`
* `%sql`

More details can be found in the `sqlmagic.ipynb`

<a id='sql_connect'></a>
## Connect a Database to the `ipython-sql` Module

In the following cell, the `ipython-sql` package is loaded with the magic command `%load_ext sql`. Note, it is not loaded with an `import` statement. `ipython-sql` supports a variety of databases. The command `%sql sqlite://` makes a connection to an in-memory SQLite database. This database is now bound to the notebook and future `%sql` command will be performed on this database.

In [ ]:
%load_ext sql
%sql sqlite://

<a id='sql_DML'></a>
## Data Manipulation Language Commands

A data manipulation language (DML) command can be issued with the `%%sql` command once a database is bound to the `ipython-sql` module. The following DML statements will create a table called `author` and populate it with three authors.

In [ ]:
%%sql
DROP TABLE IF EXISTS author;
CREATE TABLE author (given_name, family_name, year_of_death);
INSERT INTO author VALUES ('William', 'Shakespeare', 1616);
INSERT INTO author VALUES ('Bertold', 'Brecht', 1956);
INSERT INTO author VALUES ('Virginia', 'Woolf', 1941);

<a id='sql_DQL'></a>
## Data Query Language Commands

A data query language (DQL) command can be used to obtain records from the database. The syntax 

```
%%sql <variable> << 
<DQL>
``` 


is used to perform a DQL operation and store the result is the variable `<variable>`.

In [ ]:
%%sql author << 
SELECT given_name, family_name, year_of_death 
FROM author;

An alternative method to store the record set of a DQL command is to use the command `<variable> = %sql <DQL>`. This is better suited for single line DQL commands.

In [ ]:
author = %sql SELECT given_name, family_name, year_of_death FROM author
author

<a id='sql_var'></a>
## Variable Bindings

Python variables can be bound to the SQL commands with the syntax `:<variable>`, `'{variable}'` or `$variable`. In the following cell, the variable `name` is set to William. Then a DML command is issued to return any results where the `given_name` is equal to the value of `name`.

In [ ]:
first_name = 'William'
last_name = 'Shakespeare'
death_century = 1600

In [ ]:
%%sql 
SELECT * 
FROM author 
WHERE 
    given_name LIKE :first_name 
    AND family_name LIKE '{last_name}'
    AND year_of_death >= CAST('$death_century' AS INT)

# References

* [ipython-sql](https://pypi.org/project/ipython-sql/)
* [Understanding and Using Conda Environments](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/use-notebook-sessions.htm#conda_understand_environments)
* [ADS Configuration documentation](https://docs.cloud.oracle.com/en-us/iaas/tools/ads-sdk/latest/user_guide/configuration/configuration.html#)
* [Authenticating to the Oracle Cloud Infrastructure APIs from a Notebook Session](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/use-notebook-sessions.htm#topic_kxj_znw_pkb)
* [Manually Configuring Your Tenancy for Data Science](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/configure-tenancy.htm)
* [Data Science documentation](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm)
* [Data Science & AI Blog](https://blogs.oracle.com/datascience/)